<a href="https://colab.research.google.com/github/Chigoziee/Chess/blob/main/Chess_piece_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import libraries
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
from PIL import Image

In [36]:
# Define transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [54]:
# Load the datasets with ImageFolder
train_dir = "/content/drive/MyDrive/Projects/Chess/Train"
valid_dir = "/content/drive/MyDrive/Projects/Chess/Validate"
test_dir = "/content/drive/MyDrive/Projects/Chess/Test"

train_datasets = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_datasets = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_datasets = datasets.ImageFolder(test_dir, transform=test_transforms)



In [55]:
#Using the image datasets and the transforms, define the dataloaders
trainloaders = torch.utils.data.DataLoader(train_datasets,
                                          batch_size=32,
                                          shuffle=True)

validloaders = torch.utils.data.DataLoader(valid_datasets,
                                          batch_size=32,
                                          shuffle=True)

testloaders = torch.utils.data.DataLoader(test_datasets,
                                          batch_size=32,
                                          shuffle=True)

In [56]:
# loading the pretrained model
model = models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [57]:
#freezing network parameters so they do not train
for param in model.parameters():
    param.requires_grad = False

#adjusting the classifier network of the pretrained model from 1000 classes into 6 classses
classifier = nn.Sequential(nn.Linear(1024, 500),
                           nn.ReLU(),
                           nn.Dropout(p=0.5),
                           nn.Linear(500, 6),
                           nn.LogSoftmax(dim=1))

model.classifier = classifier
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [58]:
criterion = nn.NLLLoss() #loss function
optimizer = optim.Adam(model.classifier.parameters(), lr=0.005) #optimizer training only classifier parameters

In [60]:
# Initialize variables to keep track of losses and epochs
running_loss = 0
validation_loss = 0
epochs = 20

# Function to perform validation on the model
def validation(model, validloaders, criterion):
    # Initialize validation loss and accuracy
    validation_loss = 0
    accuracy = 0

    # Iterate over validation data
    for images, labels in validloaders:
        # Forward pass
        output = model.forward(images)
        # Compute validation loss
        validation_loss += criterion(output, labels).item()
        # Compute accuracy
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()

    return validation_loss, accuracy


# Function to train the model
def train(model, validloaders, trainloaders, optimizer, criterion, epochs, lr=0.005):

    # Iterate over epochs
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        running_loss = 0
        # Iterate over training data
        for images, labels in trainloaders:

            optimizer.zero_grad()

            # Forward pass
            output = model.forward(images)
            # Compute loss
            loss = criterion(output, labels)
            # Backpropagation
            loss.backward()
            optimizer.step()
            # Update running loss
            running_loss += loss.item()

        # Change model to evaluation mode
        model.eval()

        with torch.no_grad():
            # Perform validation
            validation_loss, accuracy = validation(model, validloaders, criterion)

        # Print epoch statistics
        print('Epoch: {}/{}..'.format(epoch+1, epochs),
              'Training Loss: {:.3f}..'.format(running_loss/len(trainloaders)),
              'Validation Loss: {:.3f}..'.format(validation_loss/len(validloaders)),
              'Validation Accuracy: {:.3f}%'.format(accuracy/len(validloaders) * 100))

    return model


In [61]:
model = train(model, validloaders, trainloaders, optimizer, criterion, epochs, lr=0.005)

Epoch: 1/20.. Training Loss: 2.704.. Validation Loss: 1.749.. Validation Accuracy: 26.042%
Epoch: 2/20.. Training Loss: 1.707.. Validation Loss: 1.554.. Validation Accuracy: 44.531%
Epoch: 3/20.. Training Loss: 1.451.. Validation Loss: 1.374.. Validation Accuracy: 50.000%
Epoch: 4/20.. Training Loss: 1.343.. Validation Loss: 1.334.. Validation Accuracy: 53.906%
Epoch: 5/20.. Training Loss: 1.261.. Validation Loss: 1.219.. Validation Accuracy: 52.604%
Epoch: 6/20.. Training Loss: 1.090.. Validation Loss: 1.408.. Validation Accuracy: 41.406%
Epoch: 7/20.. Training Loss: 1.106.. Validation Loss: 1.101.. Validation Accuracy: 57.812%
Epoch: 8/20.. Training Loss: 1.033.. Validation Loss: 1.035.. Validation Accuracy: 61.198%
Epoch: 9/20.. Training Loss: 1.011.. Validation Loss: 1.294.. Validation Accuracy: 49.219%
Epoch: 10/20.. Training Loss: 0.999.. Validation Loss: 1.067.. Validation Accuracy: 59.115%
Epoch: 11/20.. Training Loss: 0.916.. Validation Loss: 1.004.. Validation Accuracy: 60.15

In [65]:
test_loss = 0
accuracy = 0
# Turn off gradients for validation, saves memory and computations
with torch.no_grad():
    for images, labels in testloaders:
        log_ps = model(images)
        test_loss += criterion(log_ps, labels)

        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))

    print("Test Loss: {:.3f}.. ".format(test_loss/len(testloaders)),
          "Test Accuracy: {:.3f}%".format(accuracy/len(testloaders)*100))

Test Loss: 1.054..  Test Accuracy: 55.469%


In [64]:
# Save the checkpoint and class_indexes
torch.save(model.state_dict(), '/content/drive/MyDrive/Projects/Chess_model/checkpoint.pth')
torch.save(trainloaders.dataset.class_to_idx, '/content/drive/MyDrive/Projects/Chess_model/class_index.pth')